# Output generation for M3G task

## Test Dataset loading

> Check thata data exists 

## **Images are very graphic, use the display option with care**

In [1]:
!ls data

images			   m3g-test-allinputs-v2      vqa_en.tsv
llava_test_en_ds_ls.json   m3g-test-allinputs-v2.zip  vqa_es2.tsv
llava_test_es_ds_ls.json   mix_json_en.json	      vqa_es3.tsv
llava_train_en_ds_ls.json  mix_json_es.json
llava_val_en_ds_ls.json    submissions


In [4]:
from torch.utils.data import Dataset
from datasets import load_dataset
from PIL import Image

class M3GDataset(Dataset):
    def __init__(self, split='train', lang="en"):
        self.data = load_dataset(
            "json",
            data_files={"test": f"data/llava_test_{lang}_ds_ls.json"},
        )[split]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return {
            "images": [Image.open(im) for im in sample["image"]], # Should be a PIL image
            "qa": [
                {
                    "question": f"{sample['conversations'][0]['value']} {sample['conversations'][1]['value']}",
                }
            ],
            "encounter_id": sample["id"],
        }

datasets_en = {
    "test": M3GDataset("test", "en"),
}

datasets_es = {
    "test": M3GDataset("test", "es"),
}

In [4]:
datasets_en["test"][0]

{'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x500>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x500>],
 'qa': [{'question': 'Take a look. Is this a skin disease? Picture 1:  On the outside of the thigh, there is a small circle of lump.  Approximately 2 months.\nPicture 2:  Small red spots on the palm.  There is slight numbness in the palm.'}],
 'encounter_id': 'ENC00908'}

In [5]:
datasets_es["test"][0]

{'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x500>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x500>],
 'qa': [{'question': 'Mirad. ¿Es una enfermedad de la piel? Foto 1: En la parte externa del muslo, hay un pequeño círculo de bultos.  Aproximadamente 2 meses.\nFoto 2: Pequeñas manchas rojas en la palma de la mano.  Ligero entumecimiento palmar.\n'}],
 'encounter_id': 'ENC00908'}

## Inferences tests

In [1]:
# common packages

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

### Baselinea, use moondream2 as a zero-shot VQA

In [2]:
# Initialize moondream. Change DEVICE to 'mps' if you're on an M1 Mac, or 'cpu' if you don't have a
# GPU. Note that fine-tuning on CPU will be very slow.

DEVICE = "cuda"
DTYPE = torch.float32 if DEVICE == "cpu" else torch.float16 # CPU doesn't support float16
MD_REVISION = "2024-03-13"

tokenizer = AutoTokenizer.from_pretrained("vikhyatk/moondream2", revision=MD_REVISION)
moondream = AutoModelForCausalLM.from_pretrained(
    "vikhyatk/moondream2", revision=MD_REVISION, trust_remote_code=True,
    attn_implementation="flash_attention_2" if DEVICE == "cuda" else None,
    torch_dtype=DTYPE, device_map={"": DEVICE}
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from IPython.display import display

sample = datasets_en['test'][20]
#display(sample['images'][0])

for qa in sample['qa']:
    print('Question:', qa['question'])
    print('Angel:', moondream.answer_question(
        moondream.encode_image(sample['images'][1]),
        qa['question'],
        tokenizer=tokenizer,
    ))

Question: Would physician please help take a look.  Is what I have erythema multiforme? Had a high fever  on August 31 and took an injection.  On September 1, the fever was not receding, and I was put on IV with saline.  I found there were numerous red rashes at the foot after I went home.  I thought it was an outbreak of Beriberi.  The rashes grew in number on September 2nd, so I went to a hospital for a check.  The physician said it was multiline erythema.   He prescribed Prednisone and two other medicines.  There was also ointment for topical use.  However they are not helping after use. Now I  have more rashes in the hands and feet, and they are rather flat, just have the feeling of the tissue beneath the skin is rotting away.  It is tender to touch.  Is it multiline erythema?
Angel: Yes, the image shows a close-up of a person's fingers with multiline erythema, which is a red rash that appears in a series of parallel lines.


### Finetuned moondream2 (training data from M3G)

In [7]:
moondream_ft = AutoModelForCausalLM.from_pretrained(
    "checkpoints/moondream-m3g-ft", revision=MD_REVISION, trust_remote_code=True,
    attn_implementation="flash_attention_2" if DEVICE == "cuda" else None,
    torch_dtype=DTYPE, device_map={"": DEVICE}
)

In [9]:
from IPython.display import display

sample = datasets_en['test'][52]
#display(sample['images'][0])

for qa in sample['qa']:
    qq = f"You are a dermatologist. What is the disease in the photo? {qa['question']}"
    print('Question:', qq)
    res = moondream_ft.answer_question(
        moondream_ft.encode_image(sample['images'][0]),
        qq,
        tokenizer=tokenizer,
        #chat_history=qa['question']
    )
    print('Agent:', res)

Question: You are a dermatologist. What is the disease in the photo? Rashes on the back side of both hands for 2 years.  Please diagnose.  Thanks!  The picture is a bit big. Patient is male, 50 years old, a tobacco farmer.  Without any obvious cause, Nodular rash grows on the back part of both hands.  It is itchy and has been there for 2 years.  It could turn better at times.  See picture.
Agent: Nodular rash.


In [12]:
from IPython.display import display

sample = datasets_en['test'][52]
#display(sample['images'][0])

for qa in sample['qa']:
    qq = f"You are a dermatologist. What is the disease in the photo? Give me the best detailed treatment for this disease."
    print('Question:', qq)
    print('Agent:', moondream_ft.answer_question(
        moondream_ft.encode_image(sample['images'][1]),
        qq,
        tokenizer=tokenizer,
        chat_history=f"{res}"
    ))

Question: You are a dermatologist. What is the disease in the photo? Give me the best detailed treatment for this disease.
Agent: It is a nodular rash.


## Build the submission

### baseline : zero-shot

In [84]:
json_sub = []

for sample in datasets['test']:
    fused_content = ""
    for im in sample["images"]:
        question = sample["qa"][0]["question"]
        qq = f"You are a dermatologist. What is the disease in the photo? Give me the best detailed treatment for this disease.\n{question}"
        res = moondream.answer_question(
                                moondream.encode_image(im),
                                qq,
                                tokenizer=tokenizer,
                                )
        fused_content = f"{fused_content} {res}"
        
        
    tmp_case = {"encounter_id": sample["encounter_id"], 
              "responses": [
                  {"content_en": f"{fused_content}"}
              ] 
              }
    json_sub.append(tmp_case)
    

In [76]:
#ARCHIVED json_sub #baseline v1

[{'encounter_id': 'ENC00908',
  'responses': [{'content_en': ' The disease in the photo is a skin infection or irritation, which is causing the red spots and bumps on the thigh. As a dermatologist, the best treatment for this condition would involve identifying the specific type of infection or irritation and prescribing an appropriate antibiotic or antifungal medication, depending on the cause. Additionally, the patient should maintain good hygiene, keep the affected area clean and dry, and avoid scratching to prevent further complications or spreading the infection. If the symptoms worsen, persistent or do not improve after a week of treatment, the patient should consult a healthcare professional for further evaluation and guidance. The disease in the photo is eczema. Eczema is a chronic skin condition characterized by dry, itchy, and inflamed skin. It can cause redness, bumps, and patches on the skin. Treatment for eczema typically involves using gentle, fragrance-free soaps, avoidi

In [8]:
import json

with open("data/submissions/run_bs_v1.json", "w") as fs:
    json.dump(json_sub, fs)

### finetuned moondream2 

In [4]:
DEVICE = "cuda"
DTYPE = torch.float32 if DEVICE == "cpu" else torch.float16 # CPU doesn't support float16
MD_REVISION = "2024-03-13"

tokenizer = AutoTokenizer.from_pretrained("vikhyatk/moondream2", revision=MD_REVISION)

moondream_ft = AutoModelForCausalLM.from_pretrained(
    "checkpoints/moondream-m3g-ft", revision=MD_REVISION, trust_remote_code=True,
    attn_implementation="flash_attention_2" if DEVICE == "cuda" else None,
    torch_dtype=DTYPE, device_map={"": DEVICE}
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
moondream_ft.eval()

json_sub = []

for sample in datasets['test']:
    fused_content = ""
    for im in sample["images"]:
        question = sample["qa"][0]["question"]
        qq = f"You are a dermatologist. What is the disease in the photo? Give me the best detailed treatment for this disease.\n{question}"
        res = moondream_ft.answer_question(
                                moondream_ft.encode_image(im),
                                qq,
                                tokenizer=tokenizer,
                                )
        fused_content = f"{fused_content} {res}"
        
        
    tmp_case = {"encounter_id": sample["encounter_id"], 
              "responses": [
                  {"content_en": f"{fused_content}"}
              ] 
              }
    json_sub.append(tmp_case)
    

In [7]:
json_sub

[{'encounter_id': 'ENC00908',
  'responses': [{'content_en': ' It should be a skin disease caused by an allergic reaction. Oral antihistamines should be taken, and topical corticosteroids should be used. It should be a skin disease caused by contact with an object.'}]},
 {'encounter_id': 'ENC00909',
  'responses': [{'content_en': ' The possibility of tinea corporis is relatively high, so a fungal examination of the skin flakes should be conducted. It is folliculitis.'}]},
 {'encounter_id': 'ENC00910',
  'responses': [{'content_en': ' It is recommended to use miconazole nitrate for tinea manus. It is recommended to use miconazole nitrate for tinea manus.'}]},
 {'encounter_id': 'ENC00911',
  'responses': [{'content_en': ' It should be eczema, in my personal opinion. It should be eczema, it would be best to check for allergens. It should be eczema, in my personal opinion.'}]},
 {'encounter_id': 'ENC00912',
  'responses': [{'content_en': ' It is recommended to first perform a pathology to 

In [8]:
import json

with open("data/submissions/run_bs_ft_v1.json", "w") as fs:
    json.dump(json_sub, fs)

### Mix approach

**We generate and intermediate file that is sent to our CORR task solution LLM to build the final response**

In [13]:
datasets_es["test"][0]

{'images': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x500>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x500>],
 'qa': [{'question': 'Take a look. Is this a skin disease? Picture 1:  On the outside of the thigh, there is a small circle of lump.  Approximately 2 months.\nPicture 2:  Small red spots on the palm.  There is slight numbness in the palm.'}],
 'encounter_id': 'ENC00908'}

In [73]:
json_mix = []

for ii, el in enumerate(json_sub):
    eid = el["encounter_id"]
    sample = {"id": eid,
              "instruction": f"Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cuál es la enfermedad y el tratamiento?",
              "input": f"CONTEXTO: {datasets_es['test'][ii]['qa'][0]['question']}\nANALISIS DE IMAGENES: {el['responses'][0]['content_en']}"
             }
    json_mix.append(sample)



In [76]:
with open("data/mix_json_es.json", "w", encoding='utf-8') as sf:
    json.dump(json_mix, sf, ensure_ascii=False)


In [77]:
json_mix

[{'id': 'ENC00908',
  'instruction': 'Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cuál es la enfermedad y el tratamiento?',
  'input': 'CONTEXTO: Mirad. ¿Es una enfermedad de la piel? Foto 1: En la parte externa del muslo, hay un pequeño círculo de bultos.  Aproximadamente 2 meses.\nFoto 2: Pequeñas manchas rojas en la palma de la mano.  Ligero entumecimiento palmar.\n\nANALISIS DE IMAGENES:  It should be a skin disease caused by an allergic reaction. Oral antihistamines should be taken, and topical corticosteroids should be used. It should be a skin disease caused by contact with an object.'},
 {'id': 'ENC00909',
  'instruction': 'Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cuál es la enfermedad y el tratamiento?',
  'input': 'CONTEXTO: Ver foto para diagnóstico. Por favor mirad las fotos.\nANALISIS DE IMAGENES:  The possibility of tinea corporis is relatively high, so a fungal examination of the skin flakes should be conducted. It is folliculitis.'},
 {'id': 'ENC00910',
  'instructio

### vqa en

The input file `vqa_LANG.tsv` is returned by the CORR LLM

In [50]:
import pandas as pd

vqa_en = pd.read_csv("data/vqa_en.tsv", delimiter='\t')

In [54]:
vqa_en

,Unnamed: 0,id,instruction,input,infer_vqa_en
0,0,ENC00908,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: Take a look. Is this a skin disease? ...,"Below is an instruction that describes a task,..."
1,1,ENC00909,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: See picture for the disease. Please s...,"Below is an instruction that describes a task,..."
2,2,ENC00910,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: How to deal with Tinea manus? During ...,"Below is an instruction that describes a task,..."
3,3,ENC00911,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: A 16 months old baby girl. Started ...,"Below is an instruction that describes a task,..."
4,4,ENC00912,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: Lichen planus Lichenoid rash is found...,"Below is an instruction that describes a task,..."
...,...,...,...,...,...
95,95,ENC01003,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: Outpatient medical case Irregular gro...,"Below is an instruction that describes a task,..."
96,96,ENC01004,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: Itchiness at upper limbs. Without any...,"Below is an instruction that describes a task,..."
97,97,ENC01005,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: See picture of the disease - maculop...,"Below is an instruction that describes a task,..."
98,98,ENC01006,Given the CONTEXT and IMAGE ANALYSIS What is t...,CONTEXT: What is seen today in physical check ...,"Below is an instruction that describes a task,..."


In [60]:
json_mix_vqa = []
for row in vqa_en[["id", "infer_vqa_en"]].to_numpy():
    json_mix_vqa.append({"encounter_id": row[0],
                        "responses":[{"content_en": row[1].split("### Response:")[1]}]})

In [61]:
json_mix_vqa

[{'encounter_id': 'ENC00908',
  'responses': [{'content_en': '\nPicture 2:  Small red spots on the palm.  There is slight numbness in the palm.'}]},
 {'encounter_id': 'ENC00909',
  'responses': [{'content_en': '\nIt is folliculitis.'}]},
 {'encounter_id': 'ENC00910',
  'responses': [{'content_en': '\nIt is recommended to use miconazole nitrate for tinea manus.'}]},
 {'encounter_id': 'ENC00911',
  'responses': [{'content_en': '\nEczema, in my personal opinion.'}]},
 {'encounter_id': 'ENC00912',
  'responses': [{'content_en': '\nIt is recommended to first perform a pathology to rule out MF.'}]},
 {'encounter_id': 'ENC00913',
  'responses': [{'content_en': '\nYou can try applying Baidang externally three times a day. It is a Chinese herbal medicine for promoting blood circulation and removing bloodstasis.'}]},
 {'encounter_id': 'ENC00914',
  'responses': [{'content_en': '\nIt is recommended to first test for fungus, then consider palmoplantar keratoderma.'}]},
 {'encounter_id': 'ENC00915'

In [62]:
with open("data/submissions/run_bs_ft_vq1_en_v1.json", "w", encoding='utf-8') as fs:
    json.dump(json_mix_vqa, fs, ensure_ascii=False)

### vqa es

In [78]:
import pandas as pd

vqa_es = pd.read_csv("data/vqa_es3.tsv", delimiter='\t')

In [79]:
vqa_es

,Unnamed: 0,id,instruction,input,infer_vqa_es
0,0,ENC00908,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: Mirad. ¿Es una enfermedad de la piel...,"Below is an instruction that describes a task,..."
1,1,ENC00909,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: Ver foto para diagnóstico. Por favor...,"Below is an instruction that describes a task,..."
2,2,ENC00910,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: ¿Cómo se trata la tiña de la mano? D...,"Below is an instruction that describes a task,..."
3,3,ENC00911,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: Niña de 16 meses. Empezó a tener e...,"Below is an instruction that describes a task,..."
4,4,ENC00912,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: Líquen plano La erupción liquenoide ...,"Below is an instruction that describes a task,..."
...,...,...,...,...,...
95,95,ENC01003,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: Paciente de consultas externas Creci...,"Below is an instruction that describes a task,..."
96,96,ENC01004,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: Picor en los brazos. Sin ninguna cau...,"Below is an instruction that describes a task,..."
97,97,ENC01005,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: Ver foto de la enfermedad - erupción...,"Below is an instruction that describes a task,..."
98,98,ENC01006,Dado el CONTEXTO y el ANALISIS DE IMAGENES ¿cu...,CONTEXTO: Hoy durante el examen físico. Solic...,"Below is an instruction that describes a task,..."


In [80]:
json_mix_vqas = []
for row in vqa_es[["id", "infer_vqa_es"]].to_numpy():
    json_mix_vqas.append({"encounter_id": row[0],
                        "responses":[{"content_es": row[1].split("### Response:")[1]}]})

In [81]:
json_mix_vqas

[{'encounter_id': 'ENC00908',
  'responses': [{'content_es': '\nEs una enfermedadad cutánea causada por una alergía. Se recomienda tomar medicamentos orales antihistamínicos y usar corticosteroides tópicos. Es'}]},
 {'encounter_id': 'ENC00909',
  'responses': [{'content_es': '\nEl diagnóstico es: tinea corporis\nTratamiento: terbinafina'}]},
 {'encounter_id': 'ENC00910',
  'responses': [{'content_es': '\nRecomi<strong>ndose el tratamiento con Pevisone.'}]},
 {'encounter_id': 'ENC00911',
  'responses': [{'content_es': '\nSi, puede ser eczema, en mi opinion personal considero que se deberia importante comprobar los agentes sensibilizas alergicas'}]},
 {'encounter_id': 'ENC00912',
  'responses': [{'content_es': '\nNo se realiza una biopsia de pueso subcutáneo de la pieza de piel'}]},
 {'encounter_id': 'ENC00913',
  'responses': [{'content_es': '\nLa opción correcta es: 1. Se aplicaría baidobang externamente tres veces al día. \nLa opción correcta es: 2. Se aplicaría rix.'}]},
 {'encounter

In [82]:
with open("data/submissions/run_bs_ft_vq1_es_v2.json", "w", encoding='utf-8') as fs:
    json.dump(json_mix_vqas, fs, ensure_ascii=False)